# WEB SCRAPING

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [2]:
def price(d):
    try:

        price = d[0].contents[0].replace("\n", "").replace("€ ","").strip()
    except:
        price = d[0].contents[0].get_text().replace("\n", "").replace("€ ","").strip()

    return price



In [3]:
def locali(d):
    local = d[1].get_text().replace("\xa0","").replace("+", "").replace("locali", "").strip()
    return local

In [4]:
def superficie(d):
    superf = d[2].get_text().replace("da ","").replace("\xa0m2","").replace("superficie","").strip()
    return superf

In [5]:
def bagni(d):
    bath = d[3].get_text().replace("\xa0","").replace("+", "").replace("bagni", "").strip()
    return bath

In [6]:
def piano(d):
    floor = d[4].get_text().replace("\xa0", "").replace("\n","").replace("T","0").replace("piano","").strip()
    return floor

In [7]:
#choose number of ann to put in the function
def scraping(n_ann):
    df_info = pd.DataFrame(index=['price', 'locali', 'superficie', 'bagni', 'piano'] )
    df_info
    c =1
    i = 1
    while c < n_ann:

        url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i)
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')

        mydivs = soup.findAll("div", {"class": "listing-item_body--content"})

        for a in mydivs:

            ann = "announcement"+ str(c)

            data = a.findAll("li", {"class":"lif__item"})

            if len(data) == 6 and c <= n_ann:
                #print(ann, url)

                pr = price(data)

                local = locali(data)

                sup = superficie(data)
                
                bathrooms = bagni(data) 

                floor = piano(data)
                
                df_info[ann] = [pr, local, sup,bathrooms,floor]

                c +=1

            else:
                pass
        i +=1

    return df_info
    
    

In [8]:
#for now, we scr
df = scraping(50)

In [ ]:
#To review the "A" value for "piano": do we sobstitute it?

In [9]:
df.to_pickle("info.pkl")